# 2023 5월 주식 데이터 리뷰

In [1]:
import numpy as np
import pandas as pd

## 주식 메타 데이터 불러오기

### 데이터 분석 방향


 보통 월간 데이터를 살펴볼 때 월 시작시 수집된 메타데이터와 월 시작, 종료시 모멘텀 데이터들을 기준으로 분석을 진행합니다.
 모멘텀은 한 달간 얼마나 상승했는지를 나타냅니다.


```python
stock_dt_table = boto3.resource('dynamodb').Table('stock_dt')
stock_meta_table = boto3.resource('dynamodb').Table('stock_meta')
stock_momentum_table = boto3.resource('dynamodb').Table('stock_momentum')
stock_sector_table = boto3.resource('dynamodb').Table('stock_sector')

meta_5_df = pd.DataFrame(stock_meta_table.query(KeyConditionExpression=boto3.dynamodb.conditions.Key('dt').eq('20230501'))['Items'])

momentum_5_df = pd.DataFrame(stock_momentum_table.query(KeyConditionExpression=boto3.dynamodb.conditions.Key('dt').eq('20230501'))['Items'])
momentum_6_df = pd.DataFrame(stock_momentum_table.query(KeyConditionExpression=boto3.dynamodb.conditions.Key('dt').eq('20230601'))['Items'])

momentum_df = pd.merge(momentum_5_df, momentum_6_df, on='code', suffixes=('_5', '_6'))
momentum_df = momentum_df[['code', 'm30_5', 'm30_6']]

df = pd.merge(meta_5_df, momentum_df, on='code')
df.RR = df.RR.astype(float)
df.ROA = df.ROA.astype(float)
df.OIGR = df.OIGR.astype(float)
df.ROE = df.ROE.astype(float)
df.PER = df.PER.astype(float)
df.PBR = df.PBR.astype(float)
df.dropna(inplace=True)
```

In [2]:

from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/jetech-trading.github/jetech-trading.github.io/stock/data/202305_stock_meta.csv')

# df = pd.read_csv('./data/202305_stock_meta.csv')

KeyboardInterrupt: 

In [32]:
df.head(5)

,RR,dt,ROA,code,OIGR,ROE,name,PER,N,PBR,m30_5,m30_6
0,1215.5,20230501,4.75,20,32.98,5.73,동화약품,11.48,547,0.64,1.009443,1.162903
1,-56.6,20230501,-8.19,40,-131.11,-19.06,KR모터스,-9.71,993,1.94,1.390396,1.001951
2,5633.7,20230501,0.35,50,-40.71,0.59,경방,65.59,484,0.35,0.986201,0.980332
3,3792.7,20230501,2.35,70,-62.38,4.01,삼양홀딩스,9.44,284,0.33,1.005753,0.955181
4,240.2,20230501,2.49,80,9.45,7.72,하이트진로,18.22,166,1.34,0.994789,1.020449


## 각 특성 데이터별 N분위 데이터로 변환

각 수치들이 실수데이터를 갖기때문에
상대적인 크기를 비교하기 위해 N분위로 표기하도록 변경합니다

In [36]:
!pip install ipympl

%matplotlib widget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.4 MB/s eta 0:00:00


In [39]:
%matplotlib widget

import numpy as np
from math import acos, pi
import matplotlib.pyplot as plt
    
vector_1 = [3,4]
vector_2 = [1,3]

In [ ]:
N = 30
df['RR_qt'] = pd.qcut(df['RR'], N, labels=False)
df['ROA_qt'] = pd.qcut(df['ROA'], N, labels=False)
df['OIGR_qt'] = pd.qcut(df['OIGR'], N, labels=False)
df['ROE_qt'] = pd.qcut(df['ROE'], N, labels=False)
df['PER_qt'] = pd.qcut(df['PER'], N, labels=False)
df['PBR_qt'] = pd.qcut(df['PBR'], N, labels=False)
df['m5_qt'] = pd.qcut(df['m30_5'], N, labels=False)
df['m6_qt'] = pd.qcut(df['m30_6'], N, labels=False)

> 3개 데이터(PER, PBR, 전달 모멘텀)의 분위별 수익률을 나타내었습니다. 밝은색의 주식의 경우 수익률이 좋았음을 나타냅니다.

In [ ]:
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.graph_objects as go

import plotly.io as pio
import plotly.express as px
import plotly.offline as py

df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species", size="sepal_length")
fig


In [ ]:


fig = make_subplots(rows=1, cols=3)
fig.add_trace(go.Scatter(x=df['PER_qt'], y=df['m5_qt'], mode='markers', marker=dict(color=df['m6_qt'], colorscale='gray'), opacity=0.5, hovertext=df['name']), row=1, col=1)
fig.add_trace(go.Scatter(x=df['PBR_qt'], y=df['m5_qt'], mode='markers', marker=dict(color=df['m6_qt'], colorscale='gray'), opacity=0.5, hovertext=df['name']), row=1, col=2)
fig.add_trace(go.Scatter(x=df['PER_qt'], y=df['PBR_qt'], mode='markers', marker=dict(color=df['m6_qt'], colorscale='gray'), opacity=0.5, hovertext=df['name']), row=1, col=3)

fig.update_xaxes(title_text="PER", row=1, col=1)
fig.update_xaxes(title_text="PBR", row=1, col=2)
fig.update_xaxes(title_text="PER", row=1, col=3)
fig.update_yaxes(title_text="Momentum 5", row=1, col=1)
fig.update_yaxes(title_text="Momentum 5", row=1, col=2)
fig.update_yaxes(title_text="PBR", row=1, col=3)

fig.update_layout(showlegend=False)
fig.show()

> 3D 그래프를 3개의 2D 그래프로 바꾸어 출력했습니다. 각 점에 여러 주식들이 겹칠 수 있고, 겹쳐진 주식의 수익률도 보일 수 있도록 투명도를 주었습니다.
> 평균적으로 띄는 색갈이 해당 분위에 해당하는 평균 수익률에 해당하는 색으로 이해하면 됩니다.
> 동일하게 색이 밝을수록 수익률이 좋음을 의미합니다.